In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv('train.csv')
df.head()

,Statement,Label
0,Says the Annies List political group supports ...,False
1,When did the decline of coal start? It started...,True
2,"Hillary Clinton agrees with John McCain ""by vo...",True
3,Health care reform legislation is likely to ma...,False
4,The economic turnaround started at the end of ...,True


In [3]:
###Drop Nan Values
df=df.dropna()

df.isnull().sum()

Statement    0
Label        0
dtype: int64

In [4]:
## Get the Independent Features

X=df.drop('Label',axis=1)

In [5]:
## Get the Dependent features
y=df['Label']

In [6]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)

In [7]:
y

array([0, 1, 1, ..., 1, 0, 0], dtype=int64)

In [8]:
X.shape

(10240, 1)

In [9]:
y.shape

(10240,)

In [29]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from keras.layers import Dropout

In [12]:
### Vocabulary size
voc_size=5000

In [13]:
messages=X.copy()

In [14]:
messages['Statement'][1]

'When did the decline of coal start? It started when natural gas took off that started to begin in (President George W.) Bushs administration.'

In [15]:
messages.reset_index(inplace=True)

In [16]:
import nltk
import re
from nltk.corpus import stopwords

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rohit\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [18]:
len(messages)

10240

In [20]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    # print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['Statement'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [22]:
# corpus

In [25]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
# onehot_repr

In [26]:
sent_length=30
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4609  561 2306]
 [   0    0    0 ... 1561 4290 2007]
 [   0    0    0 ... 4018 1038 3124]
 ...
 [   0    0    0 ... 2810  224 3636]
 [   0    0    0 ...  501 2301 2752]
 [   0    0    0 ...  314 2891 4551]]


In [27]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0, 4158, 1918,
       1969, 3743, 2004,   79, 4300, 4609,  561, 2306])

In [30]:
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 30, 40)            200000    
                                                                 
 bidirectional_1 (Bidirectio  (None, 200)              112800    
 nal)                                                            
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [31]:
len(embedded_docs),y.shape

(10240, (10240,))

In [32]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [33]:
X_final.shape,y_final.shape

((10240, 30), (10240,))

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [36]:
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
108/108 [==============================] - 9s 56ms/step - loss: 0.6843 - accuracy: 0.5587 - val_loss: 0.6649 - val_accuracy: 0.6030
Epoch 2/10
108/108 [==============================] - 6s 52ms/step - loss: 0.6323 - accuracy: 0.6436 - val_loss: 0.6660 - val_accuracy: 0.5905
Epoch 3/10
108/108 [==============================] - 6s 57ms/step - loss: 0.5613 - accuracy: 0.7137 - val_loss: 0.6876 - val_accuracy: 0.5784
Epoch 4/10
108/108 [==============================] - 10s 92ms/step - loss: 0.5063 - accuracy: 0.7580 - val_loss: 0.7608 - val_accuracy: 0.5799
Epoch 5/10
108/108 [==============================] - 11s 102ms/step - loss: 0.4336 - accuracy: 0.8015 - val_loss: 0.8476 - val_accuracy: 0.5698
Epoch 6/10
108/108 [==============================] - 5s 46ms/step - loss: 0.3767 - accuracy: 0.8350 - val_loss: 0.9851 - val_accuracy: 0.5666
Epoch 7/10
108/108 [==============================] - 5s 46ms/step - loss: 0.3281 - accuracy: 0.8569 - val_loss: 1.1277 - val_accuracy: 0.5

In [38]:
predict_x=model1.predict(X_test)
y_pred=np.argmax(predict_x,axis=1)


106/106 [==============================] - 2s 8ms/step


In [39]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test,y_pred)


array([[1456,    0],
       [1924,    0]], dtype=int64)

In [40]:

from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.4307692307692308

In [41]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.4307692307692308